In [1]:
import json

In [ ]:
# Initialize the JSON structure
jsonld_data = {
    "@context": [
        "https://cns-iu.github.io/hra-cell-type-populations-supporting-information/data-processor/ccf-context.jsonld",
        {
            "UBERON": {
                "@id": "http://purl.obolibrary.org/obo/UBERON_",
                "@prefix": true
            },
            "illustration_files": {
                "@id": "ccf:has_illustration_file",
                "@type": "@id"
            },
            "mapping": {
                "@id": "ccf:has_illustration_node",
                "@type": "@id"
            },
            "organ_id": {
                "@id": "ccf:organ_id",
                "@type": "@id"
            },
            "data_sources": {
                "@id": "ccf:has_data_source",
                "@type": "@id"
            }
        }
    ],
    "@graph": []
}

In [ ]:
for cell_type in df1['cell_type'].unique():
    cell_summary = {
        "@type": "CellSummary",
        "cell_source": "Datasource file",  # Update this with actual data source if available
        "annotation_method": "Aggregation",
        "biomarker_type": "gene",
        "summary": []
    }
    
    # Fetch CL_ID for the cell_type from df1
    cl_id = df1[df1['cell_type'] == cell_type]['CL_ID'].iloc[0]
    
    genes_data = df2_merged[df2_merged['cell_type'] == cell_type]
    for _, row in genes_data.iterrows():
        gene_data = {
            "@type": "GeneExpression",
            "ensembl_id": row['gene'],
            "gene_id": row['HGNC_ID'],
            "gene_label": row['HGNC_Symbol'],
            "mean_expression": row['normalized_mean_expression'],
            "p_val": row['pvals'],
            "log2fc": row['log2fc'],
            "pvals_adj": row['pvals_adj']
        }
        cell_summary["summary"].append({
            "@type": "CellSummaryRow",
            "cell_id": cl_id,
            "cell_label": cell_type,
            "genes": [gene_data]
        })
    
    jsonld_data["@graph"].append(cell_summary)

# Save the JSON-LD data to a file
with open('cell_data.jsonld', 'w') as outfile:
    json.dump(jsonld_data, outfile, indent=4)

print("JSON-LD file has been created.")